In [46]:
import pandas as pd
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import praw
import pickle

In [47]:
def save_pickle(file_name,obj):
    with open(file_name, 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return file_name + ' is saved'

def open_pickle(file_name):
    with open(file_name, 'rb') as handle:
        obj = pickle.load(handle)
    return obj

In [51]:
reddit_cred = open_pickle('reddit_credentials.pkl')

In [117]:
reddit = praw.Reddit(client_id=reddit_cred['client_id'],
                     client_secret=reddit_cred['client_secret'],
                     redirect_uri="http://localhost:8889",
                     user_agent=reddit_cred['user_agent'],
                     username=reddit_cred['username'],
                     password=reddit_cred['password'])

In [118]:
print(reddit.auth.url(["identity"], "...", "permanent"))

https://www.reddit.com/api/v1/authorize?client_id=HFXjHXfgkYhJJA&duration=permanent&redirect_uri=http%3A%2F%2Flocalhost%3A8889&response_type=code&scope=identity&state=...


In [120]:
print(reddit.user.me())

moel9002


In [68]:
subreddit = reddit.subreddit('CallofDuty')

In [69]:
rd = {'title':subreddit.title,
      'id':subreddit.id,'url':subreddit.url,
      'subscribers':subreddit.subscribers
      }

In [75]:
import time
start = time.time()
time.sleep(5)
time.time() - start

5.002120018005371

In [133]:
def get_credentials(path):
    
    with open(path, 'rb') as handle:
         cred = pickle.load(handle)
    
    return cred
    
    
def retrieve_comment_and_post_count(reddit_data_dict,subreddit):
    lookback_hours = 24

    headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
        }
    stats = {'comment':'num_of_comments', 'submission':'num_of_posts'}
    url = "https://api.pushshift.io/reddit/{}/search?&limit=1000&sort=desc&subreddit={}&before="

    startTime = datetime.datetime.utcnow()
    endTime = startTime - timedelta(hours=lookback_hours)
    endEpoch = int(endTime.timestamp())

    for stat in stats:
        count = 0
        breakOut = False
        previousEpoch = int(startTime.timestamp())
        while True:
            newUrl = url.format(stat, subreddit)+str(previousEpoch)
            json = requests.get(newUrl, headers=headers)
            objects = json.json()['data']
            for obj in objects:
                previousEpoch = obj['created_utc'] - 1
                if previousEpoch < endEpoch:
                    breakOut = True
                    break
                count += 1

            if breakOut:
                reddit_data_dict[stats[stat]].append(count)
                break

def parse_recent_activity(current_timestamp, credentials, subreddit_names,reddit_data):
    now = datetime.datetime.now().strftime("%Y_%m_%d")
    file_name = 'subreddit_scrape_' + now + '_dict.pkl'
    
    
    reddit = praw.Reddit(client_id=credentials['client_id'],
                         client_secret=credentials['client_secret'],
                         user_agent=credentials['user_agent'],
                         username=credentials['username'],
                         password=credentials['password'])
    
#     redirect_uri="http://localhost:8080",
    
    for sub in subreddit_names:
        print(sub)
        #rate limit is one request per second (60 requests per min)
        start_time = time.time()
        subreddit = reddit.subreddit(sub)
        reddit_data['date'].append(current_timestamp)
        reddit_data['title'].append(subreddit.title)
        reddit_data['id'].append(subreddit.id)
        reddit_data['subscribers'].append(subreddit.subscribers)
        retrieve_comment_and_post_count(reddit_data,subreddit)
        save_pickle(file_name, reddit_data)
        sleep_time = 5 - (time.time() - start_time)
        if sleep_time < 0:
            sleep_time = 0.5
        time.sleep(sleep_time)
        
        
    return reddit_data

def loop_through_data(current_timestamp, credentials, subreddit_names):
    
    reddit_data = {
        'date':[],
        'title':[],
        'id':[],
        'subscribers':[],
        'num_of_comments':[],
        'num_of_posts':[] 
    }
    
    for ind in list(range(0,subreddit_names.shape[0]+20,20)):
        subreddit_names[ind:ind+1]
        
    

In [147]:
df_new.subreddit.unique().shape[0]

906

[0,
 20,
 40,
 60,
 80,
 100,
 120,
 140,
 160,
 180,
 200,
 220,
 240,
 260,
 280,
 300,
 320,
 340,
 360,
 380,
 400,
 420,
 440,
 460,
 480,
 500,
 520,
 540,
 560,
 580,
 600,
 620,
 640,
 660,
 680,
 700,
 720,
 740,
 760,
 780,
 800,
 820,
 840,
 860,
 880,
 900,
 920]

In [137]:
df = open_pickle('data/all_game_titles_and_subreddit_names_030120.pkl')
df_new = open_pickle('data/game_and_subreddit_pairing_05072020.pkl')

In [140]:
df_new.subreddit

(1280,)

In [136]:
df.head()

,Game_name,Subreddit_names
0,Anthem,"[AnthemTheGame, LowSodiumAnthem, AnthemMemes]"
1,ARK: Survival Evolved,"[ARK, playark, ARK_pc, ARKone]"
2,Assassins Creed III,"[assassinscreed, assassinscreed3]"
3,Batman: Arkham City,"[arkhamgames, BatmanArkham]"
4,Batman: Arkham Knight,"[arkhamknight, arkhamgames, BatmanArkham]"


In [90]:
game_dict = {'game':[],'subreddit':[]}
for i,row in df.iterrows():
    if row['Subreddit_names'] != None:
        for sub in row['Subreddit_names']:
            game_dict['game'].append(row['Game_name'])
            game_dict['subreddit'].append(sub)
    else:
        game_dict['game'].append(row['Game_name'])
        game_dict['subreddit'].append(row['Subreddit_names'])

In [93]:
df_new = pd.DataFrame(game_dict)
save_pickle('data/game_and_subreddit_pairing_05072020.pkl',df_new)

'data/game_and_subreddit_pairing_05072020.pkl is saved'

In [131]:
reddit_cred = open_pickle('reddit_credentials.pkl')

In [132]:
reddit_cred

{'client_id': 'HFXjHXfgkYhJJA',
 'client_secret': 'aci4x2WD2Ne59d7VV8E2qpeRKAs',
 'user_agent': 'subreddit_subscribers_tester_1',
 'username': 'moel9002',
 'password': 'Esm523100RINS!'}

In [ ]:
reddit = praw.Reddit(client_id="my client id",
                     client_secret="my client secret",
                     user_agent="my user agent",
                     username="my username",
                     password="my password")

In [134]:
import datetime
now = datetime.datetime.now()
current_timestamp = now.strftime("%Y-%m-%d")
credentials = get_credentials('reddit_credentials.pkl')

subreddit_names = df_new['subreddit']

parse_recent_activity(current_timestamp, credentials, subreddit_names)

AnthemTheGame
LowSodiumAnthem
AnthemMemes
ARK
playark
ARK_pc
ARKone
assassinscreed
assassinscreed3
arkhamgames
BatmanArkham
arkhamknight
batmanarkhamorigins
Battleborn
Battlefield1_PS4
battlefield_live
battlefield_comp
battlefield_one
battlefield1
Battlefield
battlefield3
battlefield_4
battlefield4


Forbidden: received 403 HTTP response

In [113]:
df_new['subreddit'].unique()

array(['AnthemTheGame', 'LowSodiumAnthem', 'AnthemMemes', 'ARK',
       'playark', 'ARK_pc', 'ARKone', 'assassinscreed', 'assassinscreed3',
       'arkhamgames', 'BatmanArkham', 'arkhamknight',
       'batmanarkhamorigins', 'Battleborn', 'Battlefield1_PS4',
       'battlefield_live', 'battlefield_comp', 'battlefield_one',
       'battlefield1', 'Battlefield', 'battlefield3', 'battlefield_4',
       'battlefield4', 'battlefieldhardline', 'BF_Hardline',
       'Battlefield_Hardline', 'BattlefieldV', 'BioshockInfinite',
       'Bioshock', None, 'bloodbornethegame', 'bloodborne',
       'bloodbornesliders', 'BloodborneLore', 'Bloodstained',
       'Borderlands', 'Borderlands2', 'borderlands',
       'BorderlandsPreSequel', 'borderlandstps', 'CodAW', 'CallOfDuty',
       'blackops2', 'blackops3', 'CoDCompetitive', 'CODGhosts', 'COD_LFG',
       'Infinitewarfare', 'MW2', 'ModernWarfareGame', 'mw4',
       'modernwarfare', 'CODModernWarfare', 'mw3', 'Cod4Remastered',
       'CallOfDutyWorldWa

In [10]:
# browser = webdriver.PhantomJS('/usr/local/bin/phantomjs')
browser = webdriver.Chrome(ChromeDriverManager().install())

Trying to download new driver from http://chromedriver.storage.googleapis.com/81.0.4044.69/chromedriver_mac64.zip
Unpack archive /Users/elena.morais/.wdm/drivers/chromedriver/81.0.4044.69/mac64/chromedriver.zip


In [22]:
browser.get('https://www.reddit.com/r/AskReddit/')
soup = BeautifulSoup(browser.page_source,"html.parser")

Get subscriber count:

In [23]:
soup.find('div',{'class':'_3XFx6CfPlg-4Usgxm0gK8R'})

<div class="_3XFx6CfPlg-4Usgxm0gK8R">27.7m</div>

get related subreddits:

In [26]:
soup.findAll('a',{'class':'_1p4TpHzWWRAkGYYw9_jU-B'})

[<a class="_1p4TpHzWWRAkGYYw9_jU-B" href="https://www.reddit.com/r/AskReddit/wiki/sidebarsubs#wiki_ask_others" rel="noopener" role="listitem" target="_blank">Ask Others</a>,
 <a class="_1p4TpHzWWRAkGYYw9_jU-B" href="https://www.reddit.com/r/AskReddit/wiki/sidebarsubs#wiki_askreddit_offshoots" rel="noopener" role="listitem" target="_blank">AskReddit Offshoots</a>,
 <a class="_1p4TpHzWWRAkGYYw9_jU-B" href="https://www.reddit.com/r/AskReddit/wiki/sidebarsubs#wiki_find_a_subreddit" rel="noopener" role="listitem" target="_blank">Find a Subreddit</a>,
 <a class="_1p4TpHzWWRAkGYYw9_jU-B" href="https://www.reddit.com/r/AskReddit/wiki/sidebarsubs#wiki_learn_something" rel="noopener" role="listitem" target="_blank">Learn Something</a>,
 <a class="_1p4TpHzWWRAkGYYw9_jU-B" href="https://www.reddit.com/r/AskReddit/wiki/sidebarsubs#wiki_meta" rel="noopener" role="listitem" target="_blank">Meta</a>,
 <a class="_1p4TpHzWWRAkGYYw9_jU-B" href="https://www.reddit.com/r/AskReddit/wiki/sidebarsubs#wiki_off

In [35]:
posts = soup.findAll('div',{'class':'t3_gdhkmi'})

In [37]:
posts = soup.findAll('div',{'class':'_1poyrkZ7g36PawDueRza-J'})

In [39]:
len(posts)

8

In [ ]:
<div class="_1oQyIsiPHYt6nx7VOmd1sz _1RYN-7H8gYctjOQeL8p2Q7 scrollerItem _3Qkp11fjcAw9I9wtLo8frE _1qftyZQ2bhqP62lbPjoGAh  Post t3_gdhhha " id="t3_gdhhha" tabindex="-1"><div class="_23h0-EcaBUorIHC-JZyh6J" style="width: 40px; border-left-width: 4px; border-left-style: solid; border-left-color: transparent;"><div class="_1E9mcoVn4MYnuBQSVDt1gC" id="vote-arrows-t3_gdhhha"><button aria-label="upvote" aria-pressed="false" class="voteButton" data-click-id="upvote" id="upvote-button-t3_gdhhha"><span class="_2q7IQ0BUOWeEZoeAxN555e _3SUsITjKNQ7Tp0Wi2jGxIM qW0l8Af61EP35WIG6vnGk _3edNsMs0PNfyQYofMNVhsG"><i class="icon icon-upvote _2Jxk822qXs4DaXwsN7yyHA"></i></span></button><div class="_1rZYMD_4xY3gRcSS3p8ODO" style="color: rgb(26, 26, 27);">•</div><button aria-label="downvote" aria-pressed="false" class="voteButton" data-click-id="downvote"><span class="_1iKd82bq_nqObFvSH1iC_Q Q0BxYHtCOJ_rNSPJMU2Y7 _2fe-KdD2OM0ciaiux-G1EL _3yQIOwaIuF6gn8db96Gu7y"><i class="icon icon-downvote ZyxIIl4FP5gHGrJDzNpUC"></i></span></button></div></div><div class="_1poyrkZ7g36PawDueRza-J _11R7M_VOgKO1RJyRSRErT3 " data-click-id="background" style="background-color: rgb(255, 255, 255); background-position: initial initial; background-repeat: initial initial;"><div class="BiNC74axuTz66dlnEgicY"><div class="cZPZhMe-UCZ8htPodMyJ5"><div class="_3AStxql1mQsrZuUIFP9xSg nU4Je7n-eSXStTBAPMYt8"><span class="_2fCzxBE1dlMh4OFc7B3Dun" style="color: rgb(120, 124, 126);">Posted by</span><div class="_2mHuuvyV9doV3zwbZPtIPG"><a class="_2tbHP6ZydRpjI44J3syuqC _23wugcdiaj44hdfugIAlnX oQctV4n0yUb0uiHDdGnmE" href="/user/fishandmacandcheese/" style="color: rgb(120, 124, 126);">u/fishandmacandcheese</a><div id="UserInfoTooltip--t3_gdhhha"></div></div><a class="_3jOxDPIQ0KaOWpzvSQo-1s" data-click-id="timestamp" href="https://www.reddit.com/r/AskReddit/comments/gdhhha/what_is_the_creepiest_thing_a_toy_like_a/" id="PostTopMeta--Created--false--t3_gdhhha" target="_blank" rel="nofollow noopener" style="color: rgb(120, 124, 126);">4 minutes ago</a></div><div class="_2wFk1qX4e1cxk8Pkw1rAHk"></div><div class="_2LeW9tc_6Fs1n7Ou8uD-70"></div></div></div><div class="_2FCtq-QzlfuN-SwVMUZMM3 _3wiKjmhpIpoTE2r5KCm2o6 t3_gdhhha"><div class="y8HYJ-y_lTUHkQIc1mdCq _2INHSNB8V5eaWp4P0rY_mE"><a data-click-id="body" class="SQnoC3ObvgnGjWt90zD9Z _2INHSNB8V5eaWp4P0rY_mE" href="/r/AskReddit/comments/gdhhha/what_is_the_creepiest_thing_a_toy_like_a/"><div class="_2SdHzo12ISmrC8H86TgSCp _3wqmjmv3tb_k-PROt7qFZe " theme="[object Object]" style="--posttitletextcolor: #444e59;"><h3 class="_eYtD2XCVieq6emjKBH3m">what is the creepiest thing a toy like a Furby/hatchimal etc has done?</h3></div></a></div><div class="_1hLrLjnE1G_RBCNcN9MVQf">
              <img alt="" src="https://www.redditstatic.com/desktop2x/img/renderTimingPixel.png" style="width: 1px; height: 1px;" onload="(__markFirstPostVisible || function(){})();">
            </div><style>
        .t3_gdhhha ._2FCtq-QzlfuN-SwVMUZMM3 {
          --postTitle-VisitedLinkColor: #aaafb4;
          --postTitleLink-VisitedLinkColor: #aaafb4;
        }
      </style></div><div class="STit0dLageRsa2yR4te_b "></div><div class="_1ixsU4oQRnNfZ91jhBU74y"><div class="_1E9mcoVn4MYnuBQSVDt1gC _2oM1YqCxIwkvwyeZamWwhW uFwpR-OdmueYZxdY_rEDX" id="vote-arrows-t3_gdhhha"><button aria-label="upvote" aria-pressed="false" class="voteButton" data-click-id="upvote"><span class="_2q7IQ0BUOWeEZoeAxN555e _3SUsITjKNQ7Tp0Wi2jGxIM qW0l8Af61EP35WIG6vnGk _3edNsMs0PNfyQYofMNVhsG"><i class="icon icon-upvote _2Jxk822qXs4DaXwsN7yyHA"></i></span></button><div class="_1rZYMD_4xY3gRcSS3p8ODO _25IkBM0rRUqWX5ZojEMAFQ" style="color: rgb(26, 26, 27);">•</div><button aria-label="downvote" aria-pressed="false" class="voteButton" data-click-id="downvote"><span class="_1iKd82bq_nqObFvSH1iC_Q Q0BxYHtCOJ_rNSPJMU2Y7 _2fe-KdD2OM0ciaiux-G1EL _3yQIOwaIuF6gn8db96Gu7y"><i class="icon icon-downvote ZyxIIl4FP5gHGrJDzNpUC"></i></span></button></div><div class="_3-miAEojrCvx_4FQ8x3P-s"><a rel="nofollow" data-click-id="comments" data-test-id="comments-page-link-num-comments" class="_1UoeAeSRhOKSNdY_h3iS1O _1Hw7tY9pMr-T1F4P1C-xNU _2qww3J5KKzsD7e5DO0BvvU" href="/r/AskReddit/comments/gdhhha/what_is_the_creepiest_thing_a_toy_like_a/"><i class="icon icon-comment _3ch9jJ0painNf41PmU4F9i _3DVrpDrMM9NLT6TlsTUMxC" role="presentation"></i><span class="FHCV02u6Cp2zYL0fhQPsO">1 comment</span></a><div class="_JRBNstMcGxbZUxrrIKXe _3yh2bniLq7bYr4BaiXowdO _1pShbCnOaF7EGWTq6IvZux _3sUJGnemgtNczijwoT8PGg" id="t3_gdhhha-share-menu"><button data-click-id="share" class="kU8ebCMnbXfjCWfqn0WPb"><i class="icon icon-share xwmljjCrovDE5C9MasZja _1GQDWqbF-wkYWbrpmOvjqJ"></i><span class="_6_44iTtZoeY6_XChKt5b0">share</span></button></div><button class="_10K5i7NW6qcm-UoCtpB3aK YszYBnnIoNY8pZ6UwCivd _3yh2bniLq7bYr4BaiXowdO _2sAFaB0tx4Hd5KxVkdUcAx _3sUJGnemgtNczijwoT8PGg"><span class="pthKOcceozMuXLYrLlbL1"><i class="icon icon-save _3n1jtdyipCtmS0HkOM1Tfd _2V4nGS1AmzWhA62lzdCu4r"></i></span><span class="_2-cXnP74241WI7fpcpfPmg">save</span></button><div class="OccjSdFd6HkHhShRg6DOl"></div><div><button aria-expanded="false" aria-haspopup="true" aria-label="more options" id="t3_gdhhha-overflow-menu" class="_2pFdCpgBihIaYh9DSMWBIu _1EbinKu2t3KjaT2gR156Qp uMPgOFYlCc5uvpa2Lbteu"><i class="icon icon-menu _2L8b_l8zFzAkWuMyZJ1_vg _38GxRFSqSC-Z2VLi5Xzkjy"></i></button></div><div class="_21pmAV9gWG6F_UKVe7YIE0"></div></div></div></div></div>

In [2]:
import urllib, json

In [15]:
url = requests.get('https://www.reddit.com/r/politics/.json')
reddit_test = BeautifulSoup(url.content, 'lxml')

In [3]:
url = requests.get('https://www.reddit.com/r/politics/')
reddit_test_2 = BeautifulSoup(url.content, 'lxml')

In [5]:
reddit_test_2.find('div',{'class':'_3XFx6CfPlg-4Usgxm0gK8R'})

In [16]:
reddit_test

<html><body><p>{"kind": "Listing", "data": {"modhash": "", "dist": 25, "children": [{"kind": "t3", "data": {"approved_at_utc": null, "subreddit": "politics", "selftext": "", "author_fullname": "t2_o6br3", "saved": false, "mod_reason_title": null, "gilded": 0, "clicked": false, "title": "Schumer to introduce 'no PR Act' to stop Trump from placing name on stimulus checks", "link_flair_richtext": [], "subreddit_name_prefixed": "r/politics", "hidden": false, "pwls": 6, "link_flair_css_class": null, "downs": 0, "thumbnail_height": 78, "hide_score": false, "name": "t3_g8zc4i", "quarantine": false, "link_flair_text_color": "dark", "author_flair_background_color": null, "subreddit_type": "public", "ups": 23499, "total_awards_received": 3, "media_embed": {}, "thumbnail_width": 140, "author_flair_template_id": null, "is_original_content": false, "user_reports": [], "secure_media": null, "is_reddit_media_domain": false, "is_meta": false, "category": null, "secure_media_embed": {}, "link_flair_tex

In [31]:
headers={'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/7046A194A'}
url = requests.get('https://www.reddit.com/r/politics/.json',headers=headers)
json = url.json()

In [47]:
len(json['data']['children'][0]

{'kind': 't3',
 'data': {'approved_at_utc': None,
  'subreddit': 'politics',
  'selftext': 'Hi Reddit,\n\nI’m Congressman Jim Himes and I represent the southwest Connecticut. I’ll be answering questions about the federal response to the coronavirus, including what Congress has done and what we need to do moving forward. I’m a member of the of the House Permanent Select Committee on Intelligence, where I serve as the Chair of the Strategic Technologies and Advanced Research (STAR) Subcommittee, the House Committee on Financial Services and am Chair Emeritus of the New Democrat Coalition. I’ll start answering questions at 3 p.m. ET. I look forward to answering your questions and shedding some light on how the government responding to this pandemic and what you can do to stay safe.\n \nYou can find me here:\n\n - Twitter: @jahimes\n - Facebook: @CongressmanJimHimes\n - Instagram: @repjimhimes\n - Site: www.himes.house.gov\n\nProof: https://i.redd.it/nybkauuf62v41.jpg',
  'author_fullname'

In [42]:
url = requests.get('https://www.reddit.com/r/politics',headers=headers)
reddit_test = BeautifulSoup(url.content, 'html.parser')

In [43]:
reddit_test

<!DOCTYPE html>
<html lang="en-US"><head><script>
          var __SUPPORTS_TIMING_API = typeof performance === 'object' && !!performance.mark && !! performance.measure && !!performance.getEntriesByType;
          function __perfMark(name) { __SUPPORTS_TIMING_API && performance.mark(name); };
          var __firstLoaded = false;
          function __markFirstPostVisible() {
            if (__firstLoaded) { return; }
            __firstLoaded = true;
            __perfMark("first_post_title_image_loaded");
          }
        </script><script>
          __perfMark('head_tag_start');
        </script><title>Politics</title><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="origin-when-cross-origin" name="referrer"/><style>
  /* http://meyerweb.com/eric/tools/css/reset/
    v2.0 | 20110126
    License: none (public domain)
  */

  html, body, div, span, applet, object, iframe,
  h1, h2, h3, h4, h5, h6, p, blockquote, pre,
  a, abbr, a

In [38]:
reddit_test

<!DOCTYPE html>
<html lang="en-US"><head><script>
          var __SUPPORTS_TIMING_API = typeof performance === 'object' && !!performance.mark && !! performance.measure && !!performance.getEntriesByType;
          function __perfMark(name) { __SUPPORTS_TIMING_API && performance.mark(name); };
          var __firstLoaded = false;
          function __markFirstPostVisible() {
            if (__firstLoaded) { return; }
            __firstLoaded = true;
            __perfMark("first_post_title_image_loaded");
          }
        </script><script>
          __perfMark('head_tag_start');
        </script><title>Politics</title><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="origin-when-cross-origin" name="referrer"/><style>
  /* http://meyerweb.com/eric/tools/css/reset/
    v2.0 | 20110126
    License: none (public domain)
  */

  html, body, div, span, applet, object, iframe,
  h1, h2, h3, h4, h5, h6, p, blockquote, pre,
  a, abbr, a

In [65]:
subreddit = 'reddeadredemption'
lookback_hours = 24

headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
    }
types = ['comment', 'submission']
url = "https://api.pushshift.io/reddit/{}/search?&limit=1000&sort=desc&subreddit={}&before="

startTime = datetime.utcnow()
endTime = startTime - timedelta(hours=lookback_hours)
endEpoch = int(endTime.timestamp())

for type in types:
    count = 0
    breakOut = False
    previousEpoch = int(startTime.timestamp())
    while True:
        newUrl = url.format(type, subreddit)+str(previousEpoch)
        json = requests.get(newUrl, headers=headers)
        objects = json.json()['data']
        for object in objects:
            previousEpoch = object['created_utc'] - 1
            if previousEpoch < endEpoch:
                breakOut = True
                break
            count += 1

        if breakOut:
            print(f"{count} {type}s")
            break

1433 comments
1588729891
1588729892
253 submissions
1588729633
1588729634
